# Object Creation: Notification Objects

This notebook demonstrates how to use the NotificationEventManager class to process notification events and create notification objects from OCED data.

### Notes:
- Update the file paths and user ID to match your data
- The notification objects track the last action (RECEIVED/READ) and its timestamp
- Each notification is linked to both the user and the day it occurred
- The extended data includes both the original OCED data and the new notification objects

## 1. Setup

In [1]:
import datetime
import sys
# Add the project root directory to the Python path
sys.path.append('..')
from concurrent.futures import ThreadPoolExecutor
from src.oced.oced_data_query import OCEDDataQuery
from src.oced.time_objects import TimeObject
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy import stats
import gc
import json
import pandas as pd
from pathlib import Path
from typing import Dict, Any
from src.oced.notification_events import NotificationEventManager

## 2. Load OCED Data

In [2]:
# Get OCED-mHealth Data from JSON File
dataQuery = OCEDDataQuery()  
oced_data_file = f"player_107631_oced_data_time_bouts.json"
data_dict = dataQuery.load_json(oced_data_file)

# Quick look at the data structure
print(f"Number of behavior events: {len(data_dict.get('behaviorEvents', []))}")
print(f"Number of objects: {len(data_dict.get('objects', []))}")

Number of behavior events: 498
Number of objects: 169


In [3]:
# Diagnostic code to inspect notification events
notification_events = [
    notif for notif in data_dict.get('behaviorEvents', [])
    if notif['behaviorEventType'] == 'notification'
]
print(len(notification_events))
notification_events

158


[{'id': 'b75e0192-027a-452e-bc60-661d42c68d79',
  'behaviorEventType': 'notification',
  'time': '2025-05-25T12:59:53.042000',
  'behaviorEventTypeAttributes': [{'name': 'action', 'value': 'READ'}],
  'relationships': [{'id': 'd360ac20-326e-4f6f-8113-baa47ef536ef',
    'type': 'object',
    'qualifier': 'source'},
   {'id': '8d47453a-6a5c-454d-806a-971b71a11d8f',
    'type': 'object',
    'qualifier': 'occurred_on'}],
  'event_type': 'behavior'},
 {'id': 'c1a5aba3-af33-4ead-a962-1e50b6010de2',
  'behaviorEventType': 'notification',
  'time': '2025-05-25T12:59:33.754000',
  'behaviorEventTypeAttributes': [{'name': 'action', 'value': 'RECEIVED'}],
  'relationships': [{'id': 'd360ac20-326e-4f6f-8113-baa47ef536ef',
    'type': 'object',
    'qualifier': 'source'},
   {'id': '8d47453a-6a5c-454d-806a-971b71a11d8f',
    'type': 'object',
    'qualifier': 'occurred_on'}],
  'event_type': 'behavior'},
 {'id': 'b88a28a0-03d0-482c-b20e-033c06043c88',
  'behaviorEventType': 'notification',
  'time

## 3. Initialize Notification Manager and Process Data

In [4]:
# Initialize the manager
notification_manager = NotificationEventManager()

# Add notification object type to the data
oced_data_with_notification_objects = notification_manager.create_notification_object_type(data_dict)

# Create notification objects and link events
extended_data, notification_objects = notification_manager.create_notification_objects(
    data=oced_data_with_notification_objects,
    user_id='e4365330-3468-48c3-bce3-145668e06f8f'
)

Notification object type added.

Found 158 notification events
Found 0 existing notification objects


Processing notification events: 100%|██████████| 158/158 [00:00<00:00, 2043.93it/s]

Creating new notification object ffab18b0-c61b-4764-a671-d83076c8b8af for received event
Linking read event to notification ffab18b0-c61b-4764-a671-d83076c8b8af (received at 2025-05-09 08:51:37.242000)
Creating new notification object 4c73181c-f5a4-413f-961f-20cd4f26cdeb for received event
Linking read event to notification 4c73181c-f5a4-413f-961f-20cd4f26cdeb (received at 2025-05-09 11:55:02.267000)
Creating new notification object fec91514-3db1-45f8-aaff-2f5c50bb7822 for received event
Linking read event to notification fec91514-3db1-45f8-aaff-2f5c50bb7822 (received at 2025-05-09 15:00:37.186000)
Linking read event to notification fec91514-3db1-45f8-aaff-2f5c50bb7822 (received at 2025-05-09 15:00:37.186000)
Creating new notification object df741374-2bda-425e-9402-1dbe15d75128 for received event
Linking read event to notification df741374-2bda-425e-9402-1dbe15d75128 (received at 2025-05-09 18:09:19.983000)
Creating new notification object 59224be1-c040-42b9-b872-919c4cd80550 for recei

## 4. Analyse results

In [5]:
# Print basic statistics
print(f"Total notification objects created: {len(notification_objects)}")

# Count notifications by last action
action_counts = {}
for obj in notification_objects:
    last_action = next(
        (attr['value'] for attr in obj['attributes'] 
         if attr['name'] == 'last_action'),
        None
    )
    action_counts[last_action] = action_counts.get(last_action, 0) + 1

print("\nNotifications by last action:")
for action, count in action_counts.items():
    print(f"- {action}: {count}")

# Optional: Create a DataFrame for more analysis
notifications_df = pd.DataFrame([
    {
        'notification_id': obj['id'],
        'last_action': next((attr['value'] for attr in obj['attributes'] 
                            if attr['name'] == 'last_action'), None),
        'last_action_time': next((attr['time'] for attr in obj['attributes'] 
                                if attr['name'] == 'last_action'), None)
    }
    for obj in notification_objects
])

# Diagnostic code to inspect notification events
day_objects = [
    object for object in data_dict.get('objects', [])
    if object['type'] == 'day'
]
print(len(day_objects))
day_objects

# Display first few notifications
notifications_df.head()



Total notification objects created: 104

Notifications by last action:
- READ: 49
- RECEIVED: 55
28


,notification_id,last_action,last_action_time
0,ffab18b0-c61b-4764-a671-d83076c8b8af,READ,2025-05-09T08:51:55.800000
1,4c73181c-f5a4-413f-961f-20cd4f26cdeb,READ,2025-05-09T11:55:11.964000
2,fec91514-3db1-45f8-aaff-2f5c50bb7822,READ,2025-05-09T17:35:18.626000
3,df741374-2bda-425e-9402-1dbe15d75128,READ,2025-05-09T19:30:36.761000
4,59224be1-c040-42b9-b872-919c4cd80550,READ,2025-05-10T09:09:33.063000


## 5. Save Processed Data

In [6]:
# Save the extended data
output_file = "player_107631_oced_data_time_bouts_notifications.json"  # Update this name as needed
notification_manager.save_extended_data(
    filename=output_file,
    extended_data=extended_data,
    compress=False  # Set to True for compressed output
)


Saving extended data with:
- 104 notification objects
- 158 notification events
- 273 total objects
- 498 total behavior events
Saved extended data to: c:\Users\20236075\OneDrive - TU Eindhoven\Documents\GitHub\GameBus-HealthBehaviorMining\notebooks\..\data\transformed\player_107631_oced_data_time_bouts_notifications.json


## 6. Example: Query Notifications for a Specific Day

In [7]:
# Get notifications for a specific day
date_str = "2025-05-11"  # Update this date
day_notifications = notification_manager.get_notifications_for_day(
    date_str=date_str,
    data=extended_data
)

print(f"\nNotifications for {date_str}:")
for notif in day_notifications:
    last_action = next((attr['value'] for attr in notif['attributes'] 
                       if attr['name'] == 'last_action'), None)
    last_action_time = next((attr['time'] for attr in notif['attributes'] 
                            if attr['name'] == 'last_action'), None)
    print(f"- ID: {notif['id']}, Last Action: {last_action} at {last_action_time}")


Notifications for 2025-05-11:
- ID: 4f44df5e-e729-4ca1-9c5f-030a3ec2ffe0, Last Action: READ at 2025-05-11T08:52:33.183000
- ID: 7b7420b5-5ebb-4b23-81a9-c64f380edb00, Last Action: RECEIVED at 2025-05-11T11:54:15.745000
- ID: 09d1215f-726a-463e-9312-53a81c999f85, Last Action: RECEIVED at 2025-05-11T11:54:16.238000
- ID: 728a4688-7b11-4c91-ad35-d612375bcbd5, Last Action: READ at 2025-05-11T13:00:00.256000
- ID: c9c9dbc3-6404-4627-ab24-382a95d36af8, Last Action: READ at 2025-05-11T14:55:12.557000
- ID: b79c9788-de4b-4f4a-b124-ccd5e8317a13, Last Action: RECEIVED at 2025-05-11T17:55:40.027000
- ID: c63805f0-923a-48a7-87dc-2bfe3d74e3ee, Last Action: RECEIVED at 2025-05-11T17:55:40.758000
- ID: d80fe93d-6a6c-457e-aab6-2ca11b3e10c1, Last Action: READ at 2025-05-11T22:35:53.998000


## 7. Example: Get Specific Notification Details

In [8]:
# Get details for a specific notification
notification_id = notification_objects[0]['id']  # Example: first notification
notification = notification_manager.get_notification_object(notification_id)

if notification:
    print(f"\nDetails for notification {notification_id}:")
    for attr in notification['attributes']:
        print(f"- {attr['name']}: {attr['value']} (at {attr['time']})")
    print("\nRelationships:")
    for rel in notification['relationships']:
        print(f"- {rel['qualifier']}: {rel['id']} ({rel['type']})")


Details for notification ffab18b0-c61b-4764-a671-d83076c8b8af:
- last_action: READ (at 2025-05-09T08:51:55.800000)

Relationships:
- occurred_on: f9efdadf-386e-46a3-b654-d9476f18d825 (object)
- received_by: e4365330-3468-48c3-bce3-145668e06f8f (object)
